# Golf Predictor 

## (0) Load Data

In [16]:
import pandas as pd
# dfRaw is raw dataset read from csv
dfRaw = pd.read_csv("../data_2015_thru_2019.csv")
dfRaw = dfRaw.drop(['RANK FOLLOWING YEAR'], axis=1) # remove rank following year col
dfRaw

,PLAYER NAME,YEAR,ROUNDS,AVERAGE,TOTAL SG:ARG,MEASURED ROUNDS,ROUNDS_x,AVERAGE_x,SG:OTT,SG:APR,SG:ARG,AVERAGE_y,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,AVERAGE_x.1,TOTAL SG:PUTTING,AVERAGE_y.1,TOTAL SG:APP,RANK AS LABEL
0,Jason Day,2015,75,0.287,16.357,57,75,1.520,0.772,0.461,0.287,2.106,120.054,86.649,33.407,0.586,33.407,0.461,26.260,A
1,Dustin Johnson,2015,75,-0.212,-11.466,54,75,1.326,0.960,0.579,-0.212,1.455,78.552,71.623,6.932,0.128,6.932,0.579,31.252,A
2,Jordan Spieth,2015,91,0.471,31.054,66,91,1.583,0.494,0.618,0.471,2.154,142.131,104.470,37.665,0.571,37.665,0.618,40.776,A
3,Henrik Stenson,2015,63,0.082,3.604,44,63,1.774,0.448,1.244,0.082,2.210,97.245,78.041,19.205,0.436,19.205,1.244,54.732,A
4,Adam Scott,2015,51,-0.195,-6.238,32,51,0.601,0.682,0.114,-0.195,0.206,6.577,19.240,-12.662,-0.396,-12.662,0.114,3.659,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,Patton Kizzire,2019,76,-0.120,-6.337,53,76,-0.740,-0.489,-0.132,-0.120,-0.098,-5.192,-39.241,34.050,0.642,34.050,-0.132,-6.987,F
632,Peter Malnati,2019,87,0.097,7.362,76,87,-0.088,-0.410,0.225,0.097,0.379,28.774,-6.693,35.469,0.467,35.469,0.225,17.084,F
633,Bronson Burgoon,2019,59,-0.074,-3.481,47,59,-0.053,0.054,-0.033,-0.074,-0.344,-16.148,-2.469,-13.679,-0.291,-13.679,-0.033,-1.540,F
634,Chris Stroud,2019,70,0.035,1.808,52,70,-0.752,-0.557,-0.229,0.035,-0.729,-37.920,-39.088,1.165,0.022,1.165,-0.229,-11.918,F


### (0.1) Load Training Data from File

In [17]:
# --------- For Now we will be using 2015-2018 as TRAINING, 2019 as TESTING --------------
dfTrain = pd.read_csv("../training.csv")
# delete rank next year column
dfTrain = dfTrain.drop(['RANK FOLLOWING YEAR'], axis=1) # remove rank following year col
dfTrain

,PLAYER NAME,YEAR,ROUNDS,AVERAGE,TOTAL SG:ARG,MEASURED ROUNDS,ROUNDS_x,AVERAGE_x,SG:OTT,SG:APR,SG:ARG,AVERAGE_y,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,AVERAGE_x.1,TOTAL SG:PUTTING,AVERAGE_y.1,TOTAL SG:APP,RANK AS LABEL
0,Jason Day,2015,75,0.287,16.357,57,75,1.520,0.772,0.461,0.287,2.106,120.054,86.649,33.407,0.586,33.407,0.461,26.260,A
1,Dustin Johnson,2015,75,-0.212,-11.466,54,75,1.326,0.960,0.579,-0.212,1.455,78.552,71.623,6.932,0.128,6.932,0.579,31.252,A
2,Jordan Spieth,2015,91,0.471,31.054,66,91,1.583,0.494,0.618,0.471,2.154,142.131,104.470,37.665,0.571,37.665,0.618,40.776,A
3,Henrik Stenson,2015,63,0.082,3.604,44,63,1.774,0.448,1.244,0.082,2.210,97.245,78.041,19.205,0.436,19.205,1.244,54.732,A
4,Adam Scott,2015,51,-0.195,-6.238,32,51,0.601,0.682,0.114,-0.195,0.206,6.577,19.240,-12.662,-0.396,-12.662,0.114,3.659,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Johnson Wagner,2018,75,0.226,12.456,55,75,0.092,-0.213,0.078,0.226,0.715,39.339,5.049,34.292,0.623,34.292,0.078,4.304,F
500,Scott Brown,2018,102,-0.175,-14.172,81,102,-0.710,-0.265,-0.270,-0.175,-0.661,-53.530,-57.498,3.967,0.049,3.967,-0.270,-21.856,F
501,Trey Mullinax,2018,75,-0.207,-12.199,59,75,-0.144,0.506,-0.442,-0.207,-0.072,-4.231,-8.472,4.243,0.072,4.243,-0.442,-26.106,F
502,Shawn Stefani,2018,76,0.013,0.809,60,76,0.141,-0.093,0.220,0.013,0.107,6.420,8.464,-2.045,-0.034,-2.045,0.220,13.227,F


### (0.2) Load Testing Data from File

In [18]:
dfTest = pd.read_csv("../testing.csv")
dfTest = dfTest.drop("RANK FOLLOWING YEAR", axis=1)
dfTest

,PLAYER NAME,YEAR,ROUNDS,AVERAGE,TOTAL SG:ARG,MEASURED ROUNDS,ROUNDS_x,AVERAGE_x,SG:OTT,SG:APR,SG:ARG,AVERAGE_y,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,AVERAGE_x.1,TOTAL SG:PUTTING,AVERAGE_y.1,TOTAL SG:APP,RANK AS LABEL
0,Dustin Johnson,2019,73,0.234,13.098,56,73,1.225,0.703,0.288,0.234,1.321,73.995,68.619,5.378,0.096,5.378,0.288,16.143,A
1,Jon Rahm,2019,75,0.073,3.863,53,75,1.170,0.692,0.405,0.073,1.505,79.788,61.994,17.793,0.336,17.793,0.405,21.449,A
2,Justin Thomas,2019,75,0.352,19.729,56,75,1.817,0.479,0.985,0.352,1.631,91.314,101.736,-10.419,-0.186,-10.419,0.985,55.176,A
3,Rory McIlroy,2019,72,0.297,16.939,57,72,2.126,1.195,0.633,0.297,2.551,145.379,121.159,24.221,0.425,24.221,0.633,36.096,A
4,Webb Simpson,2019,81,0.294,20.269,69,81,0.899,-0.007,0.613,0.294,1.507,104.007,62.051,41.957,0.608,41.957,0.613,42.285,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,Patton Kizzire,2019,76,-0.120,-6.337,53,76,-0.740,-0.489,-0.132,-0.120,-0.098,-5.192,-39.241,34.050,0.642,34.050,-0.132,-6.987,F
128,Peter Malnati,2019,87,0.097,7.362,76,87,-0.088,-0.410,0.225,0.097,0.379,28.774,-6.693,35.469,0.467,35.469,0.225,17.084,F
129,Bronson Burgoon,2019,59,-0.074,-3.481,47,59,-0.053,0.054,-0.033,-0.074,-0.344,-16.148,-2.469,-13.679,-0.291,-13.679,-0.033,-1.540,F
130,Chris Stroud,2019,70,0.035,1.808,52,70,-0.752,-0.557,-0.229,0.035,-0.729,-37.920,-39.088,1.165,0.022,1.165,-0.229,-11.918,F


### 0.2.2 K-Fold

In [19]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.get_n_splits()
trainSets = []
testSets = []
for k in kf.split(dfRaw):
    trainSets.append(dfRaw.iloc[k[0]])
    testSets.append(dfRaw.iloc[k[1]])

### (0.3) Find Feature Names

In [20]:
featureNames = dfTrain.columns[3:-1] # Skip ID, name, year, rounds, and rank as label
nFeatures = featureNames.shape[0]
print("Number of features:", nFeatures)
featureNames

Number of features: 16


Index(['AVERAGE', 'TOTAL SG:ARG', 'MEASURED ROUNDS', 'ROUNDS_x', 'AVERAGE_x',
       'SG:OTT', 'SG:APR', 'SG:ARG', 'AVERAGE_y', 'TOTAL SG:T', 'TOTAL SG:T2G',
       'TOTAL SG:P', 'AVERAGE_x.1', 'TOTAL SG:PUTTING', 'AVERAGE_y.1',
       'TOTAL SG:APP'],
      dtype='object')

### (0.4) Select Training Instances (features and values) from the Training Data

In [21]:
trainingInstancesList = []
for train in trainSets:
    trainingInstances_x = []
    trainingInstances_y = []
    for instance in train.to_numpy():
        featureValues = list(instance[3:-1])
        label = instance[-1]
        trainingInstances_x.append(featureValues)
        trainingInstances_y.append(label)
    nTrainingInstances = len(trainingInstances_x)
    trainingInstancesList.append((trainingInstances_x, trainingInstances_y, nTrainingInstances))
    print("Number of training instances:", nTrainingInstances)

Number of training instances: 508
Number of training instances: 509
Number of training instances: 509
Number of training instances: 509
Number of training instances: 509


### (0.5) Select Test Instances from the Data

In [22]:
testInstancesList = []
for test in testSets:    
    testInstances_x = []
    testInstances_y = []
    for instance in test.to_numpy():
        featureValues = list(instance[3:-1])
        label = instance[-1]
        testInstances_x.append(featureValues)
        testInstances_y.append(label)
    nTrainingInstances = len(trainingInstances_x)
    print("Number of training instances:", nTrainingInstances)
    testInstancesList.append((testInstances_x, testInstances_y, nTrainingInstances))

Number of training instances: 509
Number of training instances: 509
Number of training instances: 509
Number of training instances: 509
Number of training instances: 509


### (0.6) MAE

In [32]:
def mae(testInstances, prediction):
    for i in range(len(prediction)):
        print('Predict: {}, Actual: {}'.format(prediction[i], testInstances[i]))

## 1. Support Vector Classification

### 1.1 Linear SVM

In [39]:
from sklearn.svm import LinearSVC
from sklearn import metrics
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = trainingInstances[0]
    testInstances_y = trainingInstances[1]
    clf = LinearSVC()
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    #print("Predicted:", y_pred)
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

C:\Users\enord\anaconda3\anaconda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\enord\anaconda3\anaconda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Accuracy: 0.2204724409448819
F1 Micro: 0.2204724409448819
F1 Macro: 0.11897081570214707
Accuracy: 0.3280943025540275
F1 Micro: 0.3280943025540275
F1 Macro: 0.21714996303018083


C:\Users\enord\anaconda3\anaconda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\enord\anaconda3\anaconda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Accuracy: 0.22789783889980353
F1 Micro: 0.22789783889980353
F1 Macro: 0.12246013584686355
Accuracy: 0.31237721021611004
F1 Micro: 0.31237721021611004
F1 Macro: 0.18251950232052927
Accuracy: 0.3280943025540275
F1 Micro: 0.3280943025540275
F1 Macro: 0.17235519800337196
Overall Accuracy:  0.2833872190337701
Overall F1 Micro:  0.2833872190337701
Overall F1 Macro:  0.16269112298061855


C:\Users\enord\anaconda3\anaconda\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


### 1.2 Kernel SVM  

Kernel is Radius Basis Function: Gaussian:  

In [40]:
from sklearn.svm import SVC
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = trainingInstances[0]
    testInstances_y = trainingInstances[1]
    clf = SVC()
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    #print("Predicted:", y_pred)
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.43503937007874016
F1 Micro: 0.43503937007874016
F1 Macro: 0.32439811238921695
Accuracy: 0.4302554027504912
F1 Micro: 0.4302554027504912
F1 Macro: 0.32674232033307804
Accuracy: 0.4302554027504912
F1 Micro: 0.4302554027504912
F1 Macro: 0.32450471760195354
Accuracy: 0.4243614931237721
F1 Micro: 0.4243614931237721
F1 Macro: 0.30992173488700225
Accuracy: 0.4106090373280943
F1 Micro: 0.41060903732809423
F1 Macro: 0.3051376319758673
Overall Accuracy:  0.4261041412063178
Overall F1 Micro:  0.4261041412063178
Overall F1 Macro:  0.31814090343742363


Kernel is Quadratic kernel ("Degree-2 polynomial kernel"):

In [41]:
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = trainingInstances[0]
    testInstances_y = trainingInstances[1]
    clf = SVC(kernel='poly', degree=2)
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    #print("Predicted:", y_pred)
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.41338582677165353
F1 Micro: 0.41338582677165353
F1 Macro: 0.30223719555434236
Accuracy: 0.40471512770137524
F1 Micro: 0.4047151277013753
F1 Macro: 0.2856861673115622
Accuracy: 0.3968565815324165
F1 Micro: 0.3968565815324165
F1 Macro: 0.28063401343017685
Accuracy: 0.4243614931237721
F1 Micro: 0.4243614931237721
F1 Macro: 0.32074883927811504
Accuracy: 0.38703339882121807
F1 Micro: 0.38703339882121807
F1 Macro: 0.2737378216930329
Overall Accuracy:  0.40527048559008705
Overall F1 Micro:  0.40527048559008716
Overall F1 Macro:  0.29260880745344586


## 2. KNN

### Training KNN Classifiers

In [42]:
# KNN WITH n_neighbors=5
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = trainingInstances[0]
    testInstances_y = trainingInstances[1]
    clf = KNeighborsClassifier() # n_neighbors=5
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    #print("Predicted:", y_pred)
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.5531496062992126
F1 Micro: 0.5531496062992126
F1 Macro: 0.5449504619016683
Accuracy: 0.5442043222003929
F1 Micro: 0.5442043222003929
F1 Macro: 0.5228823054940632
Accuracy: 0.5540275049115914
F1 Micro: 0.5540275049115914
F1 Macro: 0.5193247659751233
Accuracy: 0.5540275049115914
F1 Micro: 0.5540275049115914
F1 Macro: 0.5244769054700745
Accuracy: 0.5540275049115914
F1 Micro: 0.5540275049115914
F1 Macro: 0.5342926671351194
Overall Accuracy:  0.5518872886468759
Overall F1 Micro:  0.5518872886468759
Overall F1 Macro:  0.5291854211952097


In [43]:
# KNN WITH n_neighbors=3
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = trainingInstances[0]
    testInstances_y = trainingInstances[1]
    clf = KNeighborsClassifier(n_neighbors=3) # n_neighbors = 3
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    #print("Predicted:", y_pred)
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.5570866141732284
F1 Micro: 0.5570866141732284
F1 Macro: 0.5463890766495294
Accuracy: 0.5658153241650294
F1 Micro: 0.5658153241650294
F1 Macro: 0.5422601340589303
Accuracy: 0.5658153241650294
F1 Micro: 0.5658153241650294
F1 Macro: 0.5427105390415438
Accuracy: 0.5677799607072691
F1 Micro: 0.5677799607072691
F1 Macro: 0.5490261150114066
Accuracy: 0.5736738703339882
F1 Micro: 0.5736738703339882
F1 Macro: 0.5582017197279826
Overall Accuracy:  0.5660342187089089
Overall F1 Micro:  0.5660342187089089
Overall F1 Macro:  0.5477175168978785


## 4. Gaussian Naive Bayes

In [44]:
from sklearn.naive_bayes import GaussianNB
acc_sum = 0
f1micro_sum = 0
f1macro_sum = 0
for trainingInstances, testInstances in zip(trainingInstancesList, testInstancesList):
    trainingInstances_x = trainingInstances[0]
    trainingInstances_y = trainingInstances[1]
    testInstances_x = trainingInstances[0]
    testInstances_y = trainingInstances[1]
    clf = GaussianNB()
    clf.fit(trainingInstances_x, trainingInstances_y)
    y_pred = clf.predict(testInstances_x)
    #print("Predicted:"print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))
    acc_sum += metrics.accuracy_score(testInstances_y, y_pred)
    print("F1 Micro:", metrics.f1_score(testInstances_y, y_pred, average='micro'))
    f1micro_sum += metrics.f1_score(testInstances_y, y_pred, average='micro')
    print("F1 Macro:", metrics.f1_score(testInstances_y, y_pred, average='macro'))
    f1macro_sum += metrics.f1_score(testInstances_y, y_pred, average='macro')
    y_pred_svm = y_pred
print("Overall Accuracy: ", acc_sum / len(testSets))
print("Overall F1 Micro: ", f1micro_sum / len(testSets))
print("Overall F1 Macro: ", f1macro_sum / len(testSets))

Accuracy: 0.37401574803149606
F1 Micro: 0.37401574803149606
F1 Macro: 0.3538459336862516
Accuracy: 0.40078585461689586
F1 Micro: 0.40078585461689586
F1 Macro: 0.37040898708417985
Accuracy: 0.38703339882121807
F1 Micro: 0.38703339882121807
F1 Macro: 0.3669947053229496
Accuracy: 0.37721021611001965
F1 Micro: 0.37721021611001965
F1 Macro: 0.34181907586351823
Accuracy: 0.3988212180746562
F1 Micro: 0.3988212180746562
F1 Macro: 0.37617516913211785
Overall Accuracy:  0.38757328713085715
Overall F1 Micro:  0.38757328713085715
Overall F1 Macro:  0.3618487742178034


# 5. Naive Solution - Random Guess

In [ ]:
import random

total_count = 0
total_correct = 0

representative_sample = ["A", "A", "B", "B", "C", "C", "D", "D", "D", "D", "E", "E", "E", "E", "E", "E", "E", "E", "E",
                        "E", "F","F", "F", "F", "F", "F", "F", "F", "F", "F"]

for index, row in dfTest.iterrows():
    pred = random.sample(representative_sample, 1)[0] # draw one from representative sample
    ans = row["RANK AS LABEL"]
    total_count = total_count + 1
    if pred == ans: # see if random draw equals acutal guess
        total_correct = total_correct + 1

print(total_correct/total_count)

# 6. Naive Solution - Use Last Year's Rank

In [ ]:
total_count = 0
total_correct = 0


for index, row in dfTest.iterrows():
    pred = ""
    last_year_rank = row["RANK LAST YEAR"] # this will not work until we add the rank last year
    if last_year_rank < 21:
        pred = "A"
    elif last_year_rank < 41:
        pred = "B"
    elif last_year_rank < 61:
        pred = "C"
    elif last_year_rank < 101:
        pred = "D"
    elif last_year_rank < 201:
        pred = "E"
    else:
        pred = "F"

    ans = row["RANK AS LABEL"]
    total_count = total_count + 1
    if pred == ans:
        total_correct = total_correct + 1

print(total_correct/total_count)